In [1]:
sc.stop()

NameError: name 'sc' is not defined

In [2]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [3]:
!hdfs dfs -ls /labs/slaba02/DO_record_per_line.json

-rw-r--r--   3 hdfs hdfs   69519728 2021-02-27 21:58 /labs/slaba02/DO_record_per_line.json


In [151]:
from pyspark.sql.types import LongType, StringType, DoubleType, StructType, StructField, IntegerType

In [152]:
schema = StructType(fields=[
    StructField("cat", StringType()),
    StructField("desc", StringType()),
    StructField("id", IntegerType()),
    StructField("lang", StringType()),
    StructField("name", StringType()),
    StructField("provider", StringType())
])

In [153]:
df = sqlContext.read.json('/labs/slaba02/DO_record_per_line.json', schema=schema)
df.show()

+--------------------+--------------------+---+----+--------------------+--------------+
|                 cat|                desc| id|lang|                name|      provider|
+--------------------+--------------------+---+----+--------------------+--------------+
|3/business_manage...|This course intro...|  4|  en|Accounting Cycle:...|Canvas Network|
|              11/law|This online cours...|  5|  en|American Counter ...|Canvas Network|
|5/computer_scienc...|This course is ta...|  6|  fr|Arithmétique: en ...|Canvas Network|
|  14/social_sciences|We live in a digi...|  7|  en|Becoming a Dynami...|Canvas Network|
|2/biology_life_sc...|This self-paced c...|  8|  en|           Bioethics|Canvas Network|
|9/humanities|15/m...|This game-based c...|  9|  en|College Foundatio...|Canvas Network|
|  14/social_sciences|What’s in your di...| 10|  en|Digital Literacies I|Canvas Network|
|  14/social_sciences|The goal of the D...| 11|  en|Digital Literacie...|Canvas Network|
|  14/social_sciences

In [154]:
df.printSchema()

root
 |-- cat: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- lang: string (nullable = true)
 |-- name: string (nullable = true)
 |-- provider: string (nullable = true)



In [155]:
from pyspark.sql.functions import lower, col
import re
regex = re.compile(u'[\w\d]{2,}', re.U)
string = df.take(1)[0].name
regex.findall(string.lower())[0]

'accounting'

In [156]:
regex.findall(df.take(1)[0].name.lower())

['accounting',
 'cycle',
 'the',
 'foundation',
 'of',
 'business',
 'measurement',
 'and',
 'reporting']

In [157]:
from pyspark.sql.functions import udf
import pandas as pd
from pyspark.sql.functions import col, pandas_udf, PandasUDFType
from pyspark.sql.types import LongType, ArrayType, StringType
from pyspark.sql.functions import lower, col
import re

regex = re.compile(u'[\w\d]{2,}', re.U)
string=df.take(1)[0].name
regex.findall(string.lower())

def ppp2(string):
     return regex.findall(string)
    

print(ppp2(df.take(1)[0].desc))

['This', 'course', 'introduces', 'the', 'basic', 'financial', 'statements', 'used', 'by', 'most', 'businesses', 'as', 'well', 'as', 'the', 'essential', 'tools', 'used', 'to', 'prepare', 'them', 'This', 'course', 'will', 'serve', 'as', 'resource', 'to', 'help', 'business', 'students', 'succeed', 'in', 'their', 'upcoming', 'university', 'level', 'accounting', 'classes', 'and', 'as', 'refresher', 'for', 'upper', 'division', 'accounting', 'students', 'who', 'are', 'struggling', 'to', 'recall', 'elementary', 'concepts', 'essential', 'to', 'more', 'advanced', 'accounting', 'topics', 'Business', 'owners', 'will', 'also', 'benefit', 'from', 'this', 'class', 'by', 'gaining', 'essential', 'skills', 'necessary', 'to', 'organize', 'and', 'manage', 'information', 'pertinent', 'to', 'operating', 'their', 'business', 'At', 'the', 'conclusion', 'of', 'the', 'class', 'students', 'will', 'understand', 'the', 'balance', 'sheet', 'income', 'statement', 'and', 'cash', 'flow', 'statement', 'They', 'will', '

In [158]:
my_udfi = udf(ppp2, returnType=StringType())

In [6]:
my_courses = [[23126, u'en', u'Compass - powerful SASS library that makes your life easier'], [21617, u'en', u'Preparing for the AP* Computer Science A Exam \u2014 Part 2'], [16627, u'es', u'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'], [11556, u'es', u'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'], [16704, u'ru', u'\u041f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0435 \u043d\u0430 Lazarus'], [13702, u'ru', u'\u041c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u0430\u044f \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u043a\u0430']]
for i in my_courses:
    print(i[0])

23126
21617
16627
11556
16704
13702


In [7]:
my_cours = df.where('id in (23126, 21617, 16627, 11556, 16704, 13702)')
my_cours.show()

+--------------------+--------------------+-----+----+--------------------+--------+
|                 cat|                desc|   id|lang|                name|provider|
+--------------------+--------------------+-----+----+--------------------+--------+
|                    | La transformació...|11556|  es|Aprendizaje Colab...|   Udemy|
|6/economics_finan...|Математическая эк...|13702|  ru|Математическая эк...|  Intuit|
|                    | Hazte más emplea...|16627|  es|Aprende Excel: Ni...|   Udemy|
|5/computer_scienc...|В курсе рассматри...|16704|  ru|Программирование ...|  Intuit|
|  5/computer_science|An introduction t...|21617|  en|Preparing for the...|     edX|
|                    | Improve your SAS...|23126|  en|Compass - powerfu...|   Udemy|
+--------------------+--------------------+-----+----+--------------------+--------+



In [8]:
my_cours.count()

6

In [9]:
from pyspark.sql.functions import split

In [144]:
from pyspark.ml.linalg import DenseVector, SparseVector

In [116]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [229]:
import pyspark.sql.functions as f

In [201]:
tokenizer = Tokenizer(inputCol="desc", outputCol="words")
wordsData = tokenizer.transform(df.where("lang='ru'"))

In [202]:
wordsData.show(5)

+--------------------+--------------------+---+----+--------------------+--------------------+--------------------+
|                 cat|                desc| id|lang|                name|            provider|               words|
+--------------------+--------------------+---+----+--------------------+--------------------+--------------------+
|  5/computer_science|Часть 1. Продвину...| 46|  ru|Дополнительные гл...|Computer Science ...|[часть, 1., продв...|
|  5/computer_science|Splay-дерево и де...| 47|  ru|Алгоритмы и струк...|Computer Science ...|[splay-дерево, и,...|
|  5/computer_science|Курс посвящён тео...| 48|  ru|Технологии хранен...|Computer Science ...|[курс, посвящён, ...|
|2/biology_life_sc...|Биоинформатика — ...| 49|  ru|Алгоритмы в биоин...|Computer Science ...|[биоинформатика, ...|
|5/computer_scienc...|Курс знакомит со ...| 50|  ru|Сложность вычисле...|Computer Science ...|[курс, знакомит, ...|
+--------------------+--------------------+---+----+--------------------

In [203]:
wordsData.count()

1231

In [204]:
wordsData=wordsData.withColumn("count", f.size("words")).where("count<>0")
wordsData.count()

1231

In [205]:
hashingTF = HashingTF(inputCol="words", outputCol="hTF", numFeatures=10000)
featurizedData = hashingTF.transform(wordsData)

idf = IDF(inputCol="hTF", outputCol="TFIDF")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

### моя база

In [206]:
schema2=df.select("id","lang","desc").schema

In [207]:
_Base = spark.createDataFrame([[23126, u'en', u'Compass - powerful SASS library that makes your life easier'], [21617, u'en', u'Preparing for the AP* Computer Science A Exam \u2014 Part 2'], [16627, u'es', u'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'], [11556, u'es', u'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'], [16704, u'ru', u'\u041f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0435 \u043d\u0430 Lazarus'], [13702, u'ru', u'\u041c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u0430\u044f \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u043a\u0430']],schema=schema2)

In [221]:
_join=_Base.select("id").join(rescaledData.select("id","lang","TFIDF"),"id", how="inner").where("id=16704")

In [222]:
_join.show()

+-----+----+--------------------+
|   id|lang|               TFIDF|
+-----+----+--------------------+
|16704|  ru|(10000,[381,1144,...|
+-----+----+--------------------+



In [223]:
join=_join.join(rescaledData.select(col("id").alias("_id"),"lang",col("TFIDF").alias("_TFIDF")),"lang", how="inner")

In [174]:
@f.udf('double')
def cos(x,y):
    return float(x.dot(y) / (x.norm(2) * y.norm(2)))

In [224]:
res=join.withColumn('cos',cos("TFIDF","_TFIDF"))

In [225]:
res.where("id=_id").orderBy("cos", ascending=False).show()

+----+-----+--------------------+-----+--------------------+---+
|lang|   id|               TFIDF|  _id|              _TFIDF|cos|
+----+-----+--------------------+-----+--------------------+---+
|  ru|16704|(10000,[381,1144,...|16704|(10000,[381,1144,...|1.0|
+----+-----+--------------------+-----+--------------------+---+



In [226]:
res=res.select("lang","id","_id","TFIDF","_TFIDF","cos").where("id<>_id").orderBy("id","cos", ascending=False)

In [227]:
res.count()

1230

In [230]:
res_out=res.select("id","_id","cos", f.row_number().over(Window.partitionBy("id").orderBy(col("cos").desc())).alias("N"))

In [231]:
res_out=res_out.where("N<=10")

In [233]:
res_out.where("id=16704").show()

+-----+-----+-------------------+---+
|   id|  _id|                cos|  N|
+-----+-----+-------------------+---+
|16704| 1228|0.13259561056914948|  1|
|16704| 1219|0.13203145826676593|  2|
|16704| 1327|0.11165396465657113|  3|
|16704|20362|0.11165396465657113|  4|
|16704| 1365|  0.096256185090638|  5|
|16704|   55|0.08723747541896949|  6|
|16704|26980|0.08546174616238522|  7|
|16704|18331|0.08433126218873696|  8|
|16704| 1236|0.08378398770773124|  9|
|16704| 8186| 0.0779310749763002| 10|
+-----+-----+-------------------+---+



In [234]:
res_out.where("id=16704").select("_id").show()

+-----+
|  _id|
+-----+
| 1228|
| 1219|
| 1327|
|20362|
| 1365|
|   55|
|26980|
|18331|
| 1236|
| 8186|
+-----+



In [235]:
results = {}

results['11556'] = [16488, 11554, 16929, 9417, 13461, 5750, 23357, 7833, 9289, 17860]

results['13702'] = [864, 21079, 792, 8123, 8083, 1033, 1041, 13057, 19925, 1217]

results['16627'] = [11431, 5372, 13021, 12247, 25010, 7296, 9470, 5687, 11575, 9598]

results['16704'] = [1228, 1219, 1327, 20362, 1365, 55, 26980, 18331, 1236, 8186]

results['21617'] = [21609, 58, 21758, 21608, 21616, 21526, 7614, 21718, 7641, 21780]

results['23126'] = [13782, 13665, 24419, 20638, 2724, 25782, 2633, 2723, 13348, 15909]

In [146]:
from pyspark.sql.types import LongType, StringType, DoubleType, StructType

In [84]:
from pyspark.sql import Window

In [109]:
import json

In [236]:
with open('lab02.json', 'w') as f:
    json.dump(results, f)